In [0]:
%pip install typing-inspect==0.8.0 typing_extensions==4.5.0 langchain-community mlflow==2.9.0 lxml==4.9.3 transformers==4.30.2 langchain==0.0.344 databricks-vectorsearch
dbutils.library.restartPython()

In [0]:
%pip install transformers==4.30.2 "unstructured[pdf,docx]==0.10.30" langchain==0.1.5 llama-index==0.9.3 databricks-vectorsearch==0.22 pydantic==1.10.9 mlflow==2.10.1
#dbutils.library.restartPython()

In [0]:
%pip install mlflow==2.10.1 lxml==4.9.3 langchain==0.1.5 databricks-vectorsearch==0.22 cloudpickle==2.2.1 databricks-sdk==0.18.0 cloudpickle==2.2.1 pydantic==2.5.2
%pip install pip mlflow[databricks]==2.10.1
#dbutils.library.restartPython()

In [0]:
%run /Workspace/Users/ashwinsingaram579@gmail.com/llm-rag-chatbot/_resources/00-init $reset_all_data=false

In [0]:
%run /Workspace/Users/ashwinsingaram579@gmail.com/llm-rag-chatbot/_resources/00-init-advanced $reset_all_data=false

In [0]:
%sql
USE CATALOG main

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS main.default.technical_docs;

In [0]:
catalog = 'main'
db = 'default'

In [0]:
# List our raw PDF docs
volume_folder = "/Volumes/main/default/technical_docs"
# Let's upload some pdf files to our volume as example. Change this with your own PDFs / docs.
#upload_pdfs_to_volume(volume_folder+"/databricks-pdf")

display(dbutils.fs.ls(volume_folder))

path,name,size,modificationTime
dbfs:/Volumes/main/default/technical_docs/hotrunnerproducthandbook_19_4-external.pdf,hotrunnerproducthandbook_19_4-external.pdf,24559011,1714612927000
dbfs:/Volumes/main/default/technical_docs/hr_ultrashot_v1-1_eng.pdf,hr_ultrashot_v1-1_eng.pdf,9338820,1714612925000
dbfs:/Volumes/main/default/technical_docs/hr_ultrasynch_v2-2_eng_202103_5007549.pdf,hr_ultrasynch_v2-2_eng_202103_5007549.pdf,6657764,1714612924000
dbfs:/Volumes/main/default/technical_docs/training-course-overview-2024.pdf,training-course-overview-2024.pdf,1159371,1714612922000


In [0]:
df = (spark.readStream
        .format('cloudFiles')
        .option('cloudFiles.format', 'BINARYFILE')
        .option("pathGlobFilter", "*.pdf")
        .load('dbfs:'+volume_folder))

# Write the data as a Delta table
(df.writeStream
  .trigger(availableNow=True)
  .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/raw_docs')
  .table('pdf_raw').awaitTermination())

In [0]:
%sql SELECT * FROM main.default.pdf_raw

path,modificationTime,length,content
dbfs:/Volumes/main/default/technical_docs/hotrunnerproducthandbook_19_4-external.pdf,2024-05-02T01:22:07Z,24559011,JVBERi0xLjcNJeLjz9MNCjQ3Mzk2IDAgb2JqDTw8L0xpbmVhcml6ZWQgMS9MIDI0NTU5MDExL08gNDczOTkvRSAxMDE5MjAwL04gMjM4L1QgMjM2MTA5NjgvSCBbIDk0NCA3MTg0XT4+DWVuZG9iag0gDQp4cmVmDQo0NzM5NiA= (truncated)
dbfs:/Volumes/main/default/technical_docs/hr_ultrashot_v1-1_eng.pdf,2024-05-02T01:22:05Z,9338820,JVBERi0xLjYNJeLjz9MNCjM1NDkgMCBvYmoNPDwvTGluZWFyaXplZCAxL0wgOTMzODgyMC9PIDM1NTEvRSAxNzA2MjIvTiA3OC9UIDkzMzcyMzgvSCBbIDc3NyAxNzQ4XT4+DWVuZG9iag0gICAgICAgDQozNTY0IDAgb2JqDTw= (truncated)
dbfs:/Volumes/main/default/technical_docs/hr_ultrasynch_v2-2_eng_202103_5007549.pdf,2024-05-02T01:22:04Z,6657764,JVBERi0xLjYNJeLjz9MNCjg4MDEgMCBvYmoNPDwvTGluZWFyaXplZCAxL0wgNjY1Nzc2NC9PIDg4MDQvRSAxNzUwMTQvTiAxNzYvVCA2NjU0Nzk2L0ggWyAxMDIwIDI3NDVdPj4NZW5kb2JqDSAgICAgDQo4ODE3IDAgb2JqDTw= (truncated)
dbfs:/Volumes/main/default/technical_docs/training-course-overview-2024.pdf,2024-05-02T01:22:02Z,1159371,JVBERi0xLjcNJeLjz9MNCjQxMSAwIG9iag08PC9MaW5lYXJpemVkIDEvTCAxMTU5MzcxL08gNDE1L0UgMzM2NzM4L04gMTYvVCAxMTUxMDMwL0ggWyAxMTE2IDg1M10+Pg1lbmRvYmoNICAgICAgICAgDQp4cmVmDQo0MTEgNDE= (truncated)


In [0]:
# For production use-case, install the libraries at your cluster level with an init script instead. 
install_ocr_on_nodes()

OCR libraries installed


In [0]:
from unstructured.partition.auto import partition
import re

def extract_doc_text(x : bytes) -> str:
  # Read files and extract the values with unstructured
  sections = partition(file=io.BytesIO(x))
  def clean_section(txt):
    txt = re.sub(r'\n', '', txt)
    return re.sub(r' ?\.', '.', txt)
  # Default split is by section of document, concatenate them all together because we want to split by sentence instead.
  return "\n".join([clean_section(s.text) for s in sections]) 

In [0]:
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index import Document, set_global_tokenizer
from transformers import AutoTokenizer
from typing import Iterator

# Reduce the arrow batch size as our PDF can be big in memory
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 10)

@pandas_udf("array<string>")
def read_as_chunk(batch_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    #set llama2 as tokenizer to match our model size (will stay below BGE 1024 limit)
    set_global_tokenizer(
      AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
    )
    #Sentence splitter from llama_index to split on sentences
    splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
    def extract_and_split(b):
      txt = extract_doc_text(b)
      nodes = splitter.get_nodes_from_documents([Document(text=txt)])
      return [n.text for n in nodes]

    for x in batch_iter:
        yield x.apply(extract_and_split)

In [0]:
from mlflow.deployments import get_deploy_client

# bge-large-en Foundation models are available using the /serving-endpoints/databricks-bge-large-en/invocations api. 
deploy_client = get_deploy_client("databricks")

## NOTE: if you change your embedding model here, make sure you change it in the query step too
embeddings = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": ["What is Apache Spark?"]})
pprint(embeddings)

In [0]:
%sql
--Note that we need to enable Change Data Feed on the table to create the index
CREATE TABLE IF NOT EXISTS databricks_pdf_documentation (
  id BIGINT GENERATED BY DEFAULT AS IDENTITY,
  url STRING,
  content STRING,
  embedding ARRAY <FLOAT>
) TBLPROPERTIES (delta.enableChangeDataFeed = true); 

In [0]:
@pandas_udf("array<float>")
def get_embedding(contents: pd.Series) -> pd.Series:
    import mlflow.deployments
    deploy_client = mlflow.deployments.get_deploy_client("databricks")
    def get_embeddings(batch):
        #Note: this will fail if an exception is thrown during embedding creation (add try/except if needed) 
        response = deploy_client.predict(endpoint="databricks-bge-large-en", inputs={"input": batch})
        return [e['embedding'] for e in response.data]

    # Splitting the contents into batches of 150 items each, since the embedding model takes at most 150 inputs per request.
    max_batch_size = 150
    batches = [contents.iloc[i:i + max_batch_size] for i in range(0, len(contents), max_batch_size)]

    # Process each batch and collect the results
    all_embeddings = []
    for batch in batches:
        all_embeddings += get_embeddings(batch.tolist())

    return pd.Series(all_embeddings)

In [0]:
import io

(spark.readStream.table('pdf_raw')
      .withColumn("content", F.explode(read_as_chunk("content")))
      .withColumn("embedding", get_embedding("content"))
      .selectExpr('path as url', 'content', 'embedding')
  .writeStream
    .trigger(availableNow=True)
    .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/pdf_chunk')
    .table('mfg_pdfs').awaitTermination())

#Let's also add our documentation web page from the simple demo (make sure you run the quickstart demo first)
if table_exists(f'{catalog}.{db}.databricks_documentation'):
  (spark.readStream.option("skipChangeCommits", "true").table('databricks_documentation') #skip changes for more stable demo
      .withColumn('embedding', get_embedding("content"))
      .select('url', 'content', 'embedding')
  .writeStream
    .trigger(availableNow=True)
    .option("checkpointLocation", f'dbfs:{volume_folder}/checkpoints/docs_chunks')
    .table('mfg_pdfs').awaitTermination())

In [0]:
%sql
SELECT * FROM mfg_pdfs WHERE url like '%.pdf'

In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient(disable_notice=True)
vector_search_endpoint_name = "vector-search-demo-endpoint_115"
vsc.create_endpoint(
    name=vector_search_endpoint_name,
    endpoint_type="STANDARD"
)

In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")

vector_search_endpoint_name = "vector-search-demo-endpoint_115"
index_fullname = "main.default.mfg_vs_index"

question = "What is training course about?"

results = vsc.get_index(vector_search_endpoint_name,index_fullname).similarity_search(
  query_text=question,
  columns=["url","content"],
  num_results=1)
docs = results.get('result', {}).get('data_array', [])
pprint(docs)

[['dbfs:/Volumes/main/default/technical_docs/training-course-overview-2024.pdf',
  '5\n'
  '2\n'
  '3\n'
  '5\n'
  '4\n'
  '6\n'
  'Duration to be verified based on the System provided by the customer. ** Training will be '
  'customized based on customer requirements.\n'
  'Training Course Overview 2024\n'
  'Page – 11\n'
  'Custom-Training\n'
  'Campus Tour We offer detailed tours of our advanced manufacturing facilities.\n'
  'Basics of Injection Molding provides the basic knowledge required to install a mold in the '
  'injection molding machine as well as the knowledge required for preparation and startup.\n'
  'Hot Runner Fundamentals covers the pros and cons of the different distribution concepts '
  '(hot-runner vs. cold-runner) and how they affect the efficiency of the system. It also goes '
  'into hot runner details includ- ing gating methods, manifold pitch layouts, actuation options, '
  'design customizations, etc.\n'
  'Basic of Polymers mediates basic knowledge about th

In [0]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate(
  input_variables = ["question"],
  template = "You are an assistant. Give a short answer to this question: {question}"
)
chat_model = ChatDatabricks(endpoint="databricks-dbrx-instruct", max_tokens = 500)

chain = (
  prompt
  | chat_model
  | StrOutputParser()
)
print(chain.invoke({"question": "What is the training course about?"}))

Based on the title you provided, the training course appears to be about learning how to use Databricks SQL to analyze data and create visualizations. The course likely covers topics such as querying data using SQL, creating tables and views, and building dashboards to visualize data. However, without more information about the specific contents of the course, this is just an educated guess.


In [0]:
prompt_with_history_str = """
Your are a Injection Molding Technical Documents chatbot. Please answer Injection Molding questions only. If you don't know or not related to Injection Molding, don't answer.

Here is a history between you and a human: {chat_history}

Now, please answer this question: {question}
"""

prompt_with_history = PromptTemplate(
  input_variables = ["chat_history", "question"],
  template = prompt_with_history_str
)

In [0]:
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

#The question is the last entry of the history
def extract_question(input):
    return input[-1]["content"]

#The history is everything before the last question
def extract_history(input):
    return input[:-1]

chain_with_history = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | prompt_with_history
    | chat_model
    | StrOutputParser()
)

print(chain_with_history.invoke({
    "messages": [
        {"role": "user", "content": "What is Hot runner?"}, 
        {"role": "assistant", "content": "They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application."}, 
        {"role": "user", "content": "Does it handle throughputs up to 1,500 g/s?"}
    ]
}))

I'm sorry for any confusion, but the question about handling throughputs up to 1,500 g/s seems to be unrelated to injection molding or hot runner systems. Hot runner systems are a type of tooling solution used in injection molding to improve part quality and volume while reducing part cost. They do not directly handle throughputs in the sense of g/s. If you have any questions related to injection molding or hot runner systems, I'd be happy to help!


In [0]:
chat_model = ChatDatabricks(endpoint="databricks-dbrx-instruct", max_tokens = 200)

is_question_about_databricks_str = """
You are classifying documents to know if this question is related with Injection molding systems in Husky company. Also answer no if the last part is inappropriate. 

Here are some examples:

Question: Knowing this followup history: What is Hot runner?, classify this question: Do you have more details?
Expected Response: Yes

Question: Knowing this followup history: What is Hot runner?, classify this question: Write me a song.
Expected Response: No

Only answer with "yes" or "no". 

Knowing this followup history: {chat_history}, classify this question: {question}
"""

is_question_about_databricks_prompt = PromptTemplate(
  input_variables= ["chat_history", "question"],
  template = is_question_about_databricks_str
)

is_about_databricks_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | is_question_about_databricks_prompt
    | chat_model
    | StrOutputParser()
)

#Returns "Yes" as this is about Databricks: 
print(is_about_databricks_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is Hot runner?"}, 
        {"role": "assistant", "content": "They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application."}, 
        {"role": "user", "content": "Does it handle throughputs up to 1,500 g/s?"}
    ]
}))

Yes.

Knowing the follow-up history, the question is asking about a specific aspect of the injection molding systems in Husky company, which is the throughput capacity. The assistant has previously provided information about Husky's hot runner systems, and the user is now asking if these systems can handle a certain amount of throughput. Therefore, the question is related to injection molding systems in Husky company.


In [0]:
#Return "no" as this isn't about Databricks
print(is_about_databricks_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is the meaning of life?"}
    ]
}))

No.


In [0]:
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
index_fullname = "main.default.mfg_vs_index"
vector_search_endpoint_name = "vector-search-demo-endpoint_115"

#Let's make sure the secret is properly setup and can access our vector search index. Check the quick-start demo for more guidance
test_demo_permissions(host, secret_scope="dbdemos", secret_key="rag_sp_token", vs_endpoint_name="vector-search-demo-endpoint_115", index_name="main.default.mfg_vs_index", embedding_endpoint_name="databricks-bge-large-en", managed_embeddings = False)

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings
from langchain.chains import RetrievalQA

os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get("dbdemos", "rag_sp_token")

embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name="vector-search-demo-endpoint_115",
        index_name="main.default.mfg_vs_index"
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="content", embedding=embedding_model, columns=["url"]
    )
    return vectorstore.as_retriever(search_kwargs={'k': 4})

retriever = get_retriever()

retrieve_document_chain = (
    itemgetter("messages") 
    | RunnableLambda(extract_question)
    | retriever
)
print(retrieve_document_chain.invoke({"messages": [{"role": "user", "content": "What is the training course about?"}]}))

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


[Document(page_content='Most of the training can be delivered in sevaral languages like English, German, French. Other language requirements must be verified prior to training scheduling.\nTraining Course Overview 2024\nPage – 3\nFoundations & Fundamentals of Injection Molding Cells\nRecommended Audience: Machine Operators, Mold-Maintenance- & Assembly-Teams\nNumber of Participants: Minimum number of participants: 3 Maximum number of participants: 5\nCourse Content This training course provides fundamental knowledge about Injection Molding Cells and their operation.\nEuipment-Interface identification and process flow understanding\nMold startup & Injection-Molding-Process • Fundamentals of Polymers • Hot-Runner fundamentals • Air supply configuration and maintenance • Mold Controller setting overview\nTraining objectives By the end of this training the participants:\nCan install and startup the mold • Understand Hot-Runner Technology • Will have basic knowledge of polymers • Are aware 

In [0]:
from langchain.schema.runnable import RunnableBranch

generate_query_to_retrieve_context_template = """
Based on the chat history below, we want you to generate a query for an external data source to retrieve relevant documents so that we can better answer the question. The query should be in natural language. The external data source uses similarity search to search for relevant documents in a vector space. So the query should be similar to the relevant documents semantically. Answer with only the query. Do not add explanation.

Chat history: {chat_history}

Question: {question}
"""

generate_query_to_retrieve_context_prompt = PromptTemplate(
  input_variables= ["chat_history", "question"],
  template = generate_query_to_retrieve_context_template
)

generate_query_to_retrieve_context_chain = (
    {
        "question": itemgetter("messages") | RunnableLambda(extract_question),
        "chat_history": itemgetter("messages") | RunnableLambda(extract_history),
    }
    | RunnableBranch(  #Augment query only when there is a chat history
      (lambda x: x["chat_history"], generate_query_to_retrieve_context_prompt | chat_model | StrOutputParser()),
      (lambda x: not x["chat_history"], RunnableLambda(lambda x: x["question"])),
      RunnableLambda(lambda x: x["question"])
    )
)

#Let's try it
output = generate_query_to_retrieve_context_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is Hot runner?"}
    ]
})
print(f"Test retriever query without history: {output}")

output = generate_query_to_retrieve_context_chain.invoke({
    "messages": [
        {"role": "user", "content": "What is Hot runner?"}, 
        {"role": "assistant", "content": "They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application."}, 
        {"role": "user", "content": "Does it handle throughputs up to 1,500 g/s?"}
    ]
})
print(f"Test retriever question, summarized with history: {output}")

Test retriever query without history: What is Hot runner?
Test retriever question, summarized with history: "Hot runner systems with throughputs up to 1,500 g/s"


In [0]:
from langchain.schema.runnable import RunnableBranch, RunnableParallel, RunnablePassthrough

question_with_history_and_context_str = """
You are a trustful assistant for Service Engineers learning baout injection molding systems manufactured by Husky company. You are answering injection molding questions related to Husky Company. If you do not know the answer to a question, you truthfully say you do not know. Read the discussion to get the context of the previous conversation. In the chat discussion, you are referred to as "system". The user is referred to as "user".

Discussion: {chat_history}

Here's some context which might or might not help you answer: {context}

Answer straight, do not repeat the question, do not start with something like: the answer to the question, do not add "AI" in front of your answer, do not say: here is the answer, do not mention the context or the question.

Based on this history and context, answer this question: {question}
"""

question_with_history_and_context_prompt = PromptTemplate(
  input_variables= ["chat_history", "context", "question"],
  template = question_with_history_and_context_str
)

def format_context(docs):
    return "\n\n".join([d.page_content for d in docs])

def extract_source_urls(docs):
    return [d.metadata["url"] for d in docs]

relevant_question_chain = (
  RunnablePassthrough() |
  {
    "relevant_docs": generate_query_to_retrieve_context_prompt | chat_model | StrOutputParser() | retriever,
    "chat_history": itemgetter("chat_history"), 
    "question": itemgetter("question")
  }
  |
  {
    "context": itemgetter("relevant_docs") | RunnableLambda(format_context),
    "sources": itemgetter("relevant_docs") | RunnableLambda(extract_source_urls),
    "chat_history": itemgetter("chat_history"), 
    "question": itemgetter("question")
  }
  |
  {
    "prompt": question_with_history_and_context_prompt,
    "sources": itemgetter("sources")
  }
  |
  {
    "result": itemgetter("prompt") | chat_model | StrOutputParser(),
    "sources": itemgetter("sources")
  }
)

irrelevant_question_chain = (
  RunnableLambda(lambda x: {"result": 'I cannot answer questions that are not about Husky products.', "sources": []})
)

branch_node = RunnableBranch(
  (lambda x: "yes" in x["question_is_relevant"].lower(), relevant_question_chain),
  (lambda x: "no" in x["question_is_relevant"].lower(), irrelevant_question_chain),
  irrelevant_question_chain
)

full_chain = (
  {
    "question_is_relevant": is_about_databricks_chain,
    "question": itemgetter("messages") | RunnableLambda(extract_question),
    "chat_history": itemgetter("messages") | RunnableLambda(extract_history),    
  }
  | branch_node
)

In [0]:
def display_chat(chat_history, response):
  def user_message_html(message):
    return f"""
      <div style="width: 90%; border-radius: 10px; background-color: #c2efff; padding: 10px; box-shadow: 2px 2px 2px #F7f7f7; margin-bottom: 10px; font-size: 14px;">
        {message}
      </div>"""
  def assistant_message_html(message):
    return f"""
      <div style="width: 90%; border-radius: 10px; background-color: #e3f6fc; padding: 10px; box-shadow: 2px 2px 2px #F7f7f7; margin-bottom: 10px; margin-left: 40px; font-size: 14px">
        <img style="float: left; width:40px; margin: -10px 5px 0px -10px" src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/robot.png?raw=true"/>
        {message}
      </div>"""
  chat_history_html = "".join([user_message_html(m["content"]) if m["role"] == "user" else assistant_message_html(m["content"]) for m in chat_history])
  answer = response["result"].replace('\n', '<br/>')
  sources_html = ("<br/><br/><br/><strong>Sources:</strong><br/> <ul>" + '\n'.join([f"""<li><a href="{s}">{s}</a></li>""" for s in response["sources"]]) + "</ul>") if response["sources"] else ""
  response_html = f"""{answer}{sources_html}"""

  displayHTML(chat_history_html + assistant_message_html(response_html))

In [0]:
import json
non_relevant_dialog = {
    "messages": [
        {"role": "user", "content": "What is Hot runner?"}, 
        {"role": "assistant", "content": "They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application."}, 
        {"role": "user", "content": "Why is the sky blue?"}
    ]
}
print(f'Testing with a non relevant question...')
response = full_chain.invoke(non_relevant_dialog)
display_chat(non_relevant_dialog["messages"], response)


Testing with a non relevant question...


What is Hot runner?
 
 
 
 They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application.
 
 
 Why is the sky blue?
 
 
 
 I cannot answer questions that are not about Husky products.

In [0]:
dialog = {
    "messages": [
        {"role": "user", "content": "What is Hot runner?"}, 
        {"role": "assistant", "content": "They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application."}, 
        {"role": "user", "content": "Does it handle throughputs up to 1,500 g/s?"}
    ]
}
print(f'Testing with relevant history and question...')
response = full_chain.invoke(dialog)
display_chat(dialog["messages"], response)

Testing with relevant history and question...


What is Hot runner?
 
 
 
 They are a complete tooling solution, helping you achieve the highest levels of part quality and part volume at the lowest part cost. And they are available in both hot tip and valve gate configurations, optimized for your specific application.
 
 
 Does it handle throughputs up to 1,500 g/s?
 
 
 
 No, it does not handle throughputs up to 1,500 g/s. The maximum throughput for the Ultra 1000 Valve Gate is 450 g/s for low to medium viscosity materials, and 200 g/s for high viscosity materials. Sources: dbfs:/Volumes/main/default/technical_docs/hotrunnerproducthandbook_19_4-external.pdf 
 dbfs:/Volumes/main/default/technical_docs/hotrunnerproducthandbook_19_4-external.pdf 
 dbfs:/Volumes/main/default/technical_docs/hotrunnerproducthandbook_19_4-external.pdf 
 dbfs:/Volumes/main/default/technical_docs/hotrunnerproducthandbook_19_4-external.pdf

In [0]:
import cloudpickle
import langchain
from mlflow.models import infer_signature

mlflow.set_registry_uri("databricks-uc")
model_name = f"{catalog}.default.dbdemos_advanced_chatbot_model"

with mlflow.start_run(run_name="dbdemos_chatbot_rag") as run:
    #Get our model signature from input/output
    output = full_chain.invoke(dialog)
    signature = infer_signature(dialog, output)

    model_info = mlflow.langchain.log_model(
        full_chain,
        loader_fn=get_retriever,  # Load the retriever with DATABRICKS_TOKEN env as secret (for authentication).
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "mlflow==" + mlflow.__version__,
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch",
            "pydantic==2.5.2 --no-binary pydantic",
            "cloudpickle=="+ cloudpickle.__version__,
            "langchainhub",
            "langchain_community"],
        input_example=dialog,
        signature=signature,
        example_no_conversion=True,
    )


2024/05/02 02:51:17 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 
2024/05/02 02:51:17 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 
2024/05/02 02:51:18 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - langchainhub (current: uninstalled, required: langchainhub)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init_

Uploading artifacts:   0%|          | 0/58 [00:00<?, ?it/s]

2024/05/02 02:51:18 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Successfully registered model 'main.rag_chatbot.dbdemos_advanced_chatbot_model'.


Uploading artifacts:   0%|          | 0/58 [00:00<?, ?it/s]

2024/05/02 02:51:22 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
Created version '1' of model 'main.rag_chatbot.dbdemos_advanced_chatbot_model'.


In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks
from operator import itemgetter

TEMPLATE = """You are a trustful assistant for Service Engineers learning about injection molding systems manufactured by Husky company. You are answering injection molding questions related to Husky Company. If you do not know the answer to a question, you truthfully say you do not know. Read the discussion to get the context of the previous conversation. In the chat discussion, you are referred to as "system". The user is referred to as "user".

Use the following pieces of context to answer the question at the end:

{context}

Question: {question}

Answer:

"""

prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "question"])

def format_context(docs):
    return "\n\n".join([d.page_content for d in docs])

def extract_source_urls(docs):
    return [d.metadata["url"] for d in docs]

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    return_source_documents=True
)

question = {
    "query": "Write a summary of training course about the hot runner systems and give sources",
    "answer": "The training course about Hot Runner Systems offered by Husky covers various aspects of the technology, design, maintenance, and optimization. The courses are designed for different audiences including Mold Design Engineers, Maintenance Technicians, and Machine Operators.\n\nThe 'Hot Runner Fundamentals' course provides an understanding of the technology, benefits, nozzle technologies, part-gating techniques, and air supply configuration and maintenance. It also introduces Husky's customer support capabilities.\n\nThe 'Designing Hot-runner Systems with HUSKY' course is for Mold Design Engineers and covers HUSKY's FEA capabilities, CAD and drawing package, introduction to HUSKY's design process and project handling, nozzle technologies, air supply configuration and maintenance, and Husky's customer support capabilities.\n\nThe 'Maintenance-Training' course is for Maintenance Technicians and Machine Operators and explains how to service and maintain HUSKY Hot-Runner Systems.\n\n\nSources:\n['dbfs:/Volumes/main/default/technical_docs/training-course-overview-2024.pdf']"
}

result = chain(question)
answer = result["result"]
sources = extract_source_urls(result["source_documents"])

formatted_response = f"Response: {answer}\nSources: {sources}"
print(formatted_response)

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Response: The training course about Hot Runner Systems offered by Husky covers various aspects of the technology, design, maintenance, and optimization. The courses are designed for different audiences, including Design Engineers, Maintenance Technicians, and Machine Operators.

The "Hot Runner Fundamentals" course provides an understanding of the technology, benefits, and different gating methods. It also covers nozzle technology, part-gating techniques, air supply configuration, and maintenance.

The "Designing Hot-runner Systems with HUSKY" course is for Design Engineers and provides insights into the layouts, rules, and calculations required to engineer a well-performing Hot-Runner System. It includes HUSKY’s FEA capabilities, CAD and drawing package, and an introduction to HUSKY’s design process and project handling.

The "Maintenance-Training" course explains how to service and maintain HUSKY Hot-Runner Systems. It includes practical and theoretical sessions on component identifi

In [0]:
from mlflow.models import infer_signature
import mlflow
import langchain

mlflow.set_registry_uri("databricks-uc")
model_name = f"{catalog}.default.advanced_working_mfg"

with mlflow.start_run(run_name="advanced_working_mfg_runV2") as run:
    signature = infer_signature(question, formatted_response)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn=get_retriever,
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "mlflow",
            "langchain",
            "langchain_community",
            "databricks-vectorsearch",
            "pydantic==2.5.2 --no-binary pydantic",
            "cloudpickle",
            "langchainhub"
        ],
        input_example=question,
        signature=signature
    )

In [0]:
pprint(input_example)

{'answer': '\n'
           'The training course offered by Husky focuses on the design, maintenance, and operation '
           'of hot runner systems. The course covers various topics, including:\n'
           '\n'
           '1. Hot Runner Fundamentals: This module covers the basics of hot runner technology, '
           'its benefits, and the different distribution concepts.\n'
           '2. Gating Styles: This module discusses the various gating styles, their impact on '
           'part quality, and the setup and operation of Husky hot runner nozzles.\n'
           '3. Air-Cirquit Requirements: This module explains the importance of compressed air '
           'quality and its impact on the production process.\n'
           '4. Hot-Runner Maintenance Training-H /-C: This hands-on training module covers the '
           'maintenance and service of Husky hot runner systems.\n'
           '5. Designing Hot-Runner Systems with Husky: This module provides insights into the '
         